In [1]:
import iot_api_client as iot
import datetime
from dateutil.tz import tzutc
import time as tm
import pandas as pd
import matplotlib.pyplot as plt
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

In [2]:
def get_token():
    YOUR_CLIENT_ID = 'LEE2MI2lNlbbZhNs2s1XBPFSXkzZAUhA'
    YOUR_CLIENT_SECRET = 'PIJWDrwgLiSYIopGGR1IjVjsgXrhNQ6R55Hy0nFf0ZZTQhAlXDw4vEppX462eKIZ'

    oauth_client = BackendApplicationClient(client_id=YOUR_CLIENT_ID)
    token_url = "https://api2.arduino.cc/iot/v1/clients/token"

    oauth = OAuth2Session(client=oauth_client)
    token = oauth.fetch_token(
        token_url=token_url,
        client_id=YOUR_CLIENT_ID,
        client_secret=YOUR_CLIENT_SECRET,
        include_client_id=True,
        audience="https://api2.arduino.cc/iot",
    )
    return token
token = get_token()

In [3]:
# Now we setup the iot-api Python client, first of all create a
# configuration object. The access token goes in the config object.
client_config = iot.Configuration(host="https://api2.arduino.cc/iot")
# client_config.debug = True
client_config.access_token = token.get("access_token")

# Create the iot-api Python client with the given configuration
client = iot.ApiClient(client_config)

# Each API model has its own wrapper, here we want to interact with
# devices, so we create a DevicesV2Api object
client_devices = iot.DevicesV2Api(client)
client_things = iot.ThingsV2Api(client)
client_properties = iot.PropertiesV2Api(client)
client_series = iot.SeriesV2Api(client)

things = client_things.things_v2_list()
thing_ids = []
for i in range(len(things)):
    if things[i].name in ['Chris code', 'Noel code']:
        thing_ids.append(things[i].id)

#   Set IDs
properties = []
for thing_id in thing_ids:
    properties.append(client_properties.properties_v2_list(thing_id))

properties

[[{'created_at': datetime.datetime(2022, 12, 10, 9, 10, 26, 154752, tzinfo=tzutc()),
   'deleted_at': None,
   'href': '/iot/v1/things/47872916-d1ed-4251-851a-e833df054e0f/properties/9779b4e9-9326-4d5b-b0b1-e439871f2cf7',
   'id': '9779b4e9-9326-4d5b-b0b1-e439871f2cf7',
   'last_value': 27.125,
   'max_value': None,
   'min_value': None,
   'name': 't_30',
   'permission': 'READ_ONLY',
   'persist': True,
   'sync_id': None,
   'tag': 2.0,
   'thing_id': '47872916-d1ed-4251-851a-e833df054e0f',
   'thing_name': 'Noel code',
   'type': 'FLOAT',
   'update_parameter': 0.0,
   'update_strategy': 'ON_CHANGE',
   'updated_at': datetime.datetime(2022, 12, 10, 9, 10, 26, 154752, tzinfo=tzutc()),
   'value_updated_at': datetime.datetime(2022, 12, 26, 14, 18, 19, 117000, tzinfo=tzutc()),
   'variable_name': 't_30'},
  {'created_at': datetime.datetime(2022, 12, 10, 9, 10, 36, 445998, tzinfo=tzutc()),
   'deleted_at': None,
   'href': '/iot/v1/things/47872916-d1ed-4251-851a-e833df054e0f/properties

In [4]:
client_config = iot.Configuration(host="https://api2.arduino.cc/iot")

client_config.access_token = token.get("access_token")

client = iot.ApiClient(client_config)

client_properties = iot.PropertiesV2Api(client)

properties = client_properties.properties_v2_list(thing_id)

name_props, id_props = ([] for i in range(2))

for i in range(len(properties)):
    name_props.append(properties[i].name)
    id_props.append(properties[i].id)

In [5]:
def get_thing_id():
    token = get_token()
    # Now we setup the iot-api Python client, first of all create a
    # configuration object. The access token goes in the config object.
    client_config = iot.Configuration(host="https://api2.arduino.cc/iot")
    # client_config.debug = True
    client_config.access_token = token.get("access_token")

    # Create the iot-api Python client with the given configuration
    client = iot.ApiClient(client_config)
    client_things = iot.ThingsV2Api(client)
    things = client_things.things_v2_list()
    thing_ids = []
    for i in range(len(things)):
        if things[i].name in ['Chris code', 'Noel code']:
            thing_ids.append(things[i].id)
    #   Set IDs
    properties = []
    for thing_id in thing_ids:
        properties.append(client_properties.properties_v2_list(thing_id))
    return thing_id, properties

def checkboxes_table(properties):
    df_propids = pd.DataFrame({
        'name':[d.name for d in properties],
        'id':[d.id for d in properties]
    })
    property_name = ['t_30','t_38','t_32','t_36','t_37','t_35','t_33','t_24','t_20','t_31','t_34','t_22','t_23','t_21']
    property_purpose = ['WW-Zirkulation Rücklauf','WW-Zirkulation Vorlauf','Vorlauf Heizungsgerät',\
        'Heizungsgerät Rücklauf','Vorlauf Wasserspeicher','Rücklauf Wasserspeicher','Küche Eltern',\
            'Vorlauf Fussbodenheizung gesamt','Fussbodenheizung Vorlauf Bad','Heizung Vorlauf im Ölkeller',\
                'Außentemperatur','Fussbodenheizung Vorlauf Wohnraum','Vorlauf Mixer Heizungsseitig','Rücklauf Mixer Heizungsseitig']
    for name, purpose in zip(property_name, property_purpose):
        row = df_propids[df_propids['name']==name].index
        df_propids.loc[row, 'purpose'] = purpose
    # Make the dictionary
    dict_propid_list = df_propids.drop('name', axis=1)\
            .rename(columns={'id':'value', 'purpose':'options'})\
                    .to_dict(orient='records')
    return df_propids, dict_propid_list

thing_id, properties_unflat = get_thing_id()
properties = [item for sublist in properties_unflat for item in sublist]
df_propids, dict_propid_list = checkboxes_table(properties)



In [6]:
from datetime import timedelta

property_id = ['9779b4e9-9326-4d5b-b0b1-e439871f2cf7', '017a526a-f846-42f4-a8eb-34166469b045']
means, stds, hours, purposes = ([] for i in range(4))
for pid in property_id:
    to_date_dict = client_properties.properties_v2_timeseries(thing_id, pid)

    data_list = to_date_dict.data
    times, values = ([] for i in range(2))

    for el in data_list:
        times.append(el.time+timedelta(hours=1))
        values.append(el.value)

    df_data = pd.DataFrame({
        'datetime':times,
        'value': values
    })
    df_data['date'] = pd.to_datetime(df_data['datetime']).dt.strftime('%d.%m.%Y')
    df_data['time'] = pd.to_datetime(df_data['datetime']).dt.strftime('%H:%M')
    df_data['hour'] = pd.to_datetime(df_data['datetime']).dt.hour + pd.to_datetime(df_data['datetime']).dt.minute/60
    df_data['hour_rounded'] = round(pd.to_datetime(df_data['datetime']).dt.hour + pd.to_datetime(df_data['datetime']).dt.minute/60)
    df_data['purpose'] = df_propids[df_propids.id==pid].purpose
    purpose = df_propids[df_propids.id==pid].purpose.values
    #  Compile means and stddevs
    for i in range(25):
        mean = df_data[df_data['hour_rounded']==i]['value'].mean()
        std = df_data[df_data['hour_rounded']==i]['value'].std()
        hours.append(i)
        means.append(mean)
        stds.append(std)
        purposes.append(purpose[0])
df_avg = pd.DataFrame({
        'purpose': purposes,
        'hour': hours,
        'mean': means,
        'std': stds,
    })

In [13]:
df_data['date'].to_list()

['12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '12.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.2022',
 '13.12.20